In [160]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder

In [175]:
df=pd.read_csv("Medical Appointment No Shows.csv")

In [176]:
df.head(10)

,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No
5,9.598513e+13,5626772,F,2016-04-27T08:36:51Z,2016-04-29T00:00:00Z,76,REPÚBLICA,0,1,0,0,0,0,No
6,7.336882e+14,5630279,F,2016-04-27T15:05:12Z,2016-04-29T00:00:00Z,23,GOIABEIRAS,0,0,0,0,0,0,Yes
7,3.449833e+12,5630575,F,2016-04-27T15:39:58Z,2016-04-29T00:00:00Z,39,GOIABEIRAS,0,0,0,0,0,0,Yes
8,5.639473e+13,5638447,F,2016-04-29T08:02:16Z,2016-04-29T00:00:00Z,21,ANDORINHAS,0,0,0,0,0,0,No
9,7.812456e+13,5629123,F,2016-04-27T12:48:25Z,2016-04-29T00:00:00Z,19,CONQUISTA,0,0,0,0,0,0,No


In [177]:
df.isnull().sum()[df.isnull().sum()>0]

Series([], dtype: int64)

In [178]:
df.select_dtypes("object").columns

Index(['Gender', 'ScheduledDay', 'AppointmentDay', 'Neighbourhood', 'No-show'], dtype='object')

In [179]:
df.shape

(110527, 14)

In [180]:
print("Total duplicate rows:", df.duplicated().sum())
duplicates = df[df.duplicated()]
print(duplicates.head())

Total duplicate rows: 0
Empty DataFrame
Columns: [PatientId, AppointmentID, Gender, ScheduledDay, AppointmentDay, Age, Neighbourhood, Scholarship, Hipertension, Diabetes, Alcoholism, Handcap, SMS_received, No-show]
Index: []


In [181]:
df["AppointmentDay"].unique()

array(['2016-04-29T00:00:00Z', '2016-05-03T00:00:00Z',
       '2016-05-10T00:00:00Z', '2016-05-17T00:00:00Z',
       '2016-05-24T00:00:00Z', '2016-05-31T00:00:00Z',
       '2016-05-02T00:00:00Z', '2016-05-30T00:00:00Z',
       '2016-05-16T00:00:00Z', '2016-05-04T00:00:00Z',
       '2016-05-19T00:00:00Z', '2016-05-12T00:00:00Z',
       '2016-05-06T00:00:00Z', '2016-05-20T00:00:00Z',
       '2016-05-05T00:00:00Z', '2016-05-13T00:00:00Z',
       '2016-05-09T00:00:00Z', '2016-05-25T00:00:00Z',
       '2016-05-11T00:00:00Z', '2016-05-18T00:00:00Z',
       '2016-05-14T00:00:00Z', '2016-06-02T00:00:00Z',
       '2016-06-03T00:00:00Z', '2016-06-06T00:00:00Z',
       '2016-06-07T00:00:00Z', '2016-06-01T00:00:00Z',
       '2016-06-08T00:00:00Z'], dtype=object)

In [182]:
df["ScheduledDay"].dtype

dtype('O')

In [183]:
df.rename(columns={'No-show': 'Appointment_Status'}, inplace=True)

In [184]:
# Convert to datetime properly
df['ScheduledDay'] = pd.to_datetime(df['ScheduledDay'], errors='coerce')
df['AppointmentDay'] = pd.to_datetime(df['AppointmentDay'], errors='coerce')

In [185]:
# Extract meaningful date features
df['ScheduledYear'] = df['ScheduledDay'].dt.year
df['ScheduledMonth'] = df['ScheduledDay'].dt.month
df['ScheduledDayOfWeek'] = df['ScheduledDay'].dt.dayofweek
df['ScheduledDayOfMonth'] = df['ScheduledDay'].dt.day

df['AppointmentMonth'] = df['AppointmentDay'].dt.month
df['AppointmentDayOfWeek'] = df['AppointmentDay'].dt.dayofweek

In [187]:
# Calculate waiting time (crucial for no-shows)
df['WaitingDays'] = (df['AppointmentDay'] - df['ScheduledDay']).dt.days

In [188]:
# Drop original date columns
df = df.drop(['ScheduledDay', 'AppointmentDay'], axis=1)

In [190]:
le = LabelEncoder()
df['Gender'] = le.fit_transform(df['Gender'])  # F=0, M=1
df['Neighbourhood'] = le.fit_transform(df['Neighbourhood'])
df['Appointment_Status'] = le.fit_transform(df['Appointment_Status'])  # No=0, Yes=1

In [191]:
df["PatientId"].unique()

array([2.98724998e+13, 5.58997777e+14, 4.26296230e+12, ...,
       7.26331493e+13, 9.96997666e+14, 1.55766317e+13])

In [192]:
df['PatientId'] = df['PatientId'].astype(str)

In [ ]:
df.sample(5)


In [ ]:
df.select_dtypes("object").columns

In [ ]:
df["ScheduledDay"].unique()

In [ ]:
df.head(5)